In [151]:
import pandas as pd 
import psycopg2 as pg
import pandas.io.sql as psql

connection = pg.connect("host='172.17.14.29' dbname=finance user=finance password='finance'")
 
    

OperationalError: FATAL:  no pg_hba.conf entry for host "172.17.33.28", user "finance", database "finance", SSL off


In [2]:

sql = """select * from 
(select  b.trx_id,
 jsonb_extract_path_text(response_data,'cdfs_response_finalize','data','transaction_id_map','8012') transaction_id,
 b.from_wallet, a.from_wallet_id, b.to_wallet, uuid,customer_no,meter_no,sms_mphone,a.transaction_id titas_id,a.amount,a.status,a.initiated_at initiated_at, a.completed_at completed_at
--jsonb_extract_path_text(response_data,'cdfs_response_finalize','data','batch_id') batch_id

from fundtransferuser.payment_titasbillpayment a join fundtransferuser.transaction_transaction b
on a.trx_id=b.trx_id
join fundtransferuser.transaction_transactionlog d
on b.id=d.transaction_id
where  date(a.initiated_at) between '2021-04-01' and '2021-04-20'
--and a.status ='success' 
) as d"""


titas= pd.read_sql_query(sql, connection)
 

In [144]:
number ='01819454850'  #agent number 1819454850	110404888514004

date = '2021-04-19'  #trx_date
meter = '0110404888514004'  #to_wallet

In [145]:
sql=("""select * from cdfsuser.fiio_wallet 
where number = %s""")


wallet_id = pd.read_sql_query(sql, connection, params=[number])
wallet_ids=wallet_id['id'][0]
wallet_ids


DatabaseError: Execution failed on sql: select * from cdfsuser.fiio_wallet 
where number = %s
terminating connection due to administrator command
server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.

unable to rollback

In [146]:
sql=("""select batch_id, transaction_id, description,amount,balance, updated_at from cdfsuser.fiio_ledger
where wallet_id= %s
and date(created_at) = %s
and amount > 50
order by created_at
""")

ledger= pd.read_sql_query(sql, connection, params=[int(wallet_ids),date])
ledger

InterfaceError: connection already closed

In [147]:
sql="""
select from_wallet, to_wallet,trx_id,amount,transaction_type,status,completed_at from fundtransferuser.transaction_transaction
where  to_wallet = %s
and date(initiated_at) = %s
--and status != 'success'
ORDER BY completed_at asc"""

ft= pd.read_sql_query(sql, connection, params=[meter,date])
ft

InterfaceError: connection already closed

In [130]:


sql = """select b.number,a.* from
(select  cif_id, sum(case when category=1 then balance else 0 end) primary_balance,
sum(case when category=2 then balance else 0 end) remi_balance,
sum(case when category=3 then balance else 0 end) salary_balance,
sum(case when category=4 then balance else 0 end) disburse_balance,
sum(case when category=5 then balance else 0 end) cashback_balance,
sum(case when category=1 then frozen_balance else 0 end) f_primary_balance,
sum(case when category=2 then frozen_balance else 0 end) f_remi_balance,
sum(case when category=3 then frozen_balance else 0 end) f_salary_balance,
sum(case when category=4 then frozen_balance else 0 end) f_disburse_balance,
sum(case when category=5 then frozen_balance else 0 end) f_cashback_balance 
from cdfsuser.fiio_wallet w 
where persona=5

group by cif_id
) a left join (select * from cdfsuser.fiio_wallet where  category=1) b
on a.cif_id=b.cif_id

where number = %s"""

wal_bal= pd.read_sql_query(sql, connection, params=[number])
wal_bal

,number,cif_id,primary_balance,remi_balance,salary_balance,disburse_balance,cashback_balance,f_primary_balance,f_remi_balance,f_salary_balance,f_disburse_balance,f_cashback_balance
0,01856412122,101322,8126.96875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
